# Optimization Task

In [2]:
# Questions:
# - Is the penalty cost incurred every day or only once?
# - What are the penalty costs of an engine that is not maintained at all?

In [3]:
# MY IDEA: define the optimization problem as a permutation
# - change individual to list of engines
# - then it's just a permutation problem
# first few engines are assigned to team 1 until it does not fit anymore
# then, the next few engines are assigned to team 2 until it does not fit anymore etc.

# Population: list of (no_individuals) containing schedules
# Individual: permutation of engines requiring maintenance within the planning horizon.
# Fitness calculation: we go through the permutation. We assign engines to the first team until it does not
#   fit anymore. Then, we go to the next team, assign the engines from the engine where we left of, to the 
#   this next team until it does not fit etc. until we've done all teams. Then, the remaining engines are
#   not maintained, so we just calculate their costs (not sure how high they should be -> see questions above).
#   I think this is an optimal solution as teams A are first, and they're always more efficient than team B.
#   Therefore, moving a team to the next team is always equally or less efficient than assigning it to the
#   current team.

# TODO:
# - experiment with GA instance parameters
# - crossover: create random number of crossoverpoints

In [4]:
# Imports
import pandas as pd
import random
import numpy as np
import pygad

In [5]:
# Set seeds
np.random.seed(42)
random.seed(42)

In [6]:
# Constants
TOTAL_NO_ENGINES = 100
T_1 = 1

In [7]:
engines = list(range(1, TOTAL_NO_ENGINES + 1))

## Loading Data

In [8]:
file_path = "./data/RUL_consultancy_predictions_A3-2.csv"

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify that the data was loaded correctly
print(df.head())

# Split the "RUL;id" column into two separate columns
df[['RUL', 'id']] = df['RUL;id'].str.split(';', expand=True)

# Convert the 'RUL' column to a dictionary with 'id' column as keys
RUL = dict(zip(df['id'].astype(int), df['RUL'].astype(int)))

  RUL;id
0  135;1
1  125;2
2   63;3
3  100;4
4  103;5


## Maintenance Teams

In [9]:
maintenance_duration = {}

# Define the values for μAj
for engine in engines:
    if 1 <= engine <= 20:
        maintenance_duration[engine] = {'A': 4, 'B': 5}  # since μBj = μAj + 1
    elif 21 <= engine <= 55:
        maintenance_duration[engine] = {'A': 3, 'B': 4}  # since μBj = μAj + 1
    elif 56 <= engine <= 80:
        maintenance_duration[engine] = {'A': 2, 'B': 3}  # since μBj = μAj + 1
    else:
        maintenance_duration[engine] = {'A': 8, 'B': 9}  # since μBj = μAj + 1

# Update the values for μBj based on conditions
for engine in engines:
    if 1 <= engine <= 25:
        maintenance_duration[engine]['B'] = maintenance_duration[engine]['A'] + 1
    elif 26 <= engine <= 70:
        maintenance_duration[engine]['B'] = maintenance_duration[engine]['A'] + 2
    else:
        maintenance_duration[engine]['B'] = maintenance_duration[engine]['A'] + 1

## Safety due date

In [10]:
safety_due_date = {id_val: T_1 + rul - 1 for id_val, rul in RUL.items()}

## Penalty costs

In [11]:
penalty_costs = {}

# Define the values for cj
for engine in range(1, 21):
    penalty_costs[engine] = 4

for engine in range(21, 31):
    penalty_costs[engine] = 3

for engine in range(31, 46):
    penalty_costs[engine] = 2

for engine in range(46, 81):
    penalty_costs[engine] = 5

for engine in range(81, 101):
    penalty_costs[engine] = 6


## Help functions

In [12]:
def cost(engine, day):
    """
    Calculates the incurred cost when an engine is not maintained by its safety due date.

    Parameters:
    - engine: number of the engine
    - day: current day

    Returns:
    - integer: incurred cost for the engine on the day
    """
    if day > safety_due_date[engine]: # Engine is maintained too late
        calculated_cost = penalty_costs[engine] * ((day - safety_due_date[engine]) ** 2)
        return min(calculated_cost, 250)
    else: # Engine is maintained on time
        return 0

In [13]:
def engines_requiring_maintenance(planning_horizon):
    """
    Return the engines that require maintenance within the planning period.

    Parameters:
    - planning_horizon: number of days for which a maintenance schedule needs to be created

    Returns:
    - list: engines requiring maintenance
    """
    engines_to_maintain = []
    for engine_id, due_date in safety_due_date.items():
        if due_date <= planning_horizon:
            engines_to_maintain.append(engine_id)
    return engines_to_maintain

In [14]:
def output_format(solution):
    """
    Transform solution to the correct output format.

    """

    return

## Genetic Algorithm

In [15]:
# Constants
PLANNING_HORIZON = 30
NO_PARENTS = 100
POPULATION_SIZE = 100
NO_GENERATIONS = 100
NO_TEAMS = 4
TEAM_TYPES = ["A", "A", "B", "B"] # List indicating which type each team has (i.e. "A" or "B")
MAX_DURATION = 5 * 60  # 5 minutes in seconds
MUTATION_PROBABILITY = 0.1  # Example value, adjust as needed
PENALTY_VALUE = 1e9
ENGINES_TO_MAINTAIN = engines_requiring_maintenance(PLANNING_HORIZON)
NO_ENGINES = len(ENGINES_TO_MAINTAIN)

In [16]:
def create_initial_population():
    """
    Creates a certain number of random individuals (i.e., maintenance schedules).
    Each individual is a permutation of the list of engines requiring maintenance.

    Returns:
    - numpy array: Population containing the random individuals (i.e., maintenance schedules).
    """
    population = np.zeros((POPULATION_SIZE, NO_ENGINES))

    # Iterate to create a certain number of individuals in the population
    for individual in range(POPULATION_SIZE):
        # Each individual is a random permutation of the list of engines requiring maintenance.
        population[individual] = ENGINES_TO_MAINTAIN.copy()
        random.shuffle(population[individual])

    return population

In [17]:
def fitness_func(ga_instance, schedule, schedule_idx):
    """
    Calculate the fitness value (i.e., total incurred costs) for a given solution.

    Args:
    - ga_instance: Instance of the GA class (required argument).
    - schedule (numpy.ndarray): Solution representing a maintenance schedule.
    - schedule_idx (int): Index of the schedule (i.e., individual) in the population (required argument).

    Returns:
    - float: Total incurred costs for the given schedule.
    """
    total_costs = 0  # Initialize total costs counter
    engine_idx = 0 # Start with the first engine in the schedule

    # Iterate through the teams
    for team in range(NO_TEAMS):
        team_type = TEAM_TYPES[team]  # Get the type of the current team (i.e., "A" or "B")
        day = 1 # Start on the first day of the team
        
        # Assign engines to this team until the team's schedule is full or there are no engines left
        while day <= PLANNING_HORIZON and engine_idx < len(schedule):
            engine = schedule[engine_idx] # Number of the engine
            no_days = maintenance_duration[engine][team_type] # Number of days it takes to maintain the engine

            if day + no_days <= PLANNING_HORIZON: # Engine fits in team's schedule
                total_costs += cost(engine, day)  # Add the incurred costs of the engine
                day += no_days # Increase day counter
                engine_idx += 1 # Go to next engine in the schedule
            else: # Engine does not fit
                break # Go to the next team

    # Add costs for engines that could not be scheduled within the available days of any team
    while engine_idx < len(schedule):
        engine = schedule[engine_idx] # Number of the engine
        total_costs += cost(engine, PLANNING_HORIZON) # Add the incurred costs of the engine
        engine_idx += 1 # Go to next engine in the schedule

    return total_costs
    

In [18]:
def order_crossover(parents, offspring_size, ga_instance):
    """
    Perform order crossover on the parents to generate offspring.

    Args:
    - parents (numpy.ndarray): Parent solutions for crossover.
    - offspring_size (tuple): Size of the offspring population.
    - ga_instance (pygad.GA): Instance of the GA class (required argument).

    Returns:
    - numpy.ndarray: Offspring population generated by order crossover.
    """
    offspring = []

    while len(offspring) < offspring_size[0]:
        # Generate two unique random indices for selecting parents
        idx1, idx2 = np.random.choice(len(parents), size=2, replace=False)

        # Select parents using the generated indices
        parent1 = parents[idx1, :].copy()
        parent2 = parents[idx2, :].copy()

        # Randomly select two crossover points
        crossover_points = np.random.choice(len(parent1), size=2, replace=False)
        crossover_point1, crossover_point2 = np.sort(crossover_points)

        # Create a copy of parents to store the offspring
        child1 = np.zeros(len(parent1))
        child2 = np.zeros(len(parent2))

        # Extract the gene segments from both parents
        segment_from_parent1 = parent1[crossover_point1:crossover_point2]
        segment_from_parent2 = parent2[crossover_point1:crossover_point2]
        
        # Transfer the gene segments to the offspring
        child1[crossover_point1:crossover_point2] = segment_from_parent2
        child2[crossover_point1:crossover_point2] = segment_from_parent1

        # Find the missing values in child1
        missing_values1 = [gene for gene in parent1 if gene not in segment_from_parent2]

        # Find the missing values in offspring2
        missing_values2 = [gene for gene in parent2 if gene not in segment_from_parent1]

        # Insert missing values into child1 at the correct positions
        idx1 = 0
        for i in range(len(child1)):
            if child1[i] == 0:
                child1[i] = missing_values1[idx1]
                idx1 += 1

        offspring.append(child1) # Add first child to offspring

        if len(offspring) >= offspring_size[0]: # Enough children have been created
            break

        # Insert missing values into child2 at the correct positions
        idx2 = 0
        for i in range(len(child2)):
            if child2[i] == 0:
                child2[i] = missing_values2[idx2]
                idx2 += 1
        
        offspring.append(child2) # Add second child to offspring
        
    return np.array(offspring) # Numpy array is the required format for the offspring

In [19]:
# Initialize the GA instance
ga_instance = pygad.GA(num_generations = NO_GENERATIONS,
                       num_parents_mating = NO_PARENTS,
                       fitness_func = fitness_func, # Our custom fitness evaluation function
                       sol_per_pop = POPULATION_SIZE,
                       parent_selection_type = "rws", # Roulette Wheel Selection with Rank-Based Fitness Values
                       crossover_type=order_crossover, # Our custom order crossover function
                       mutation_type = "swap", # Use swap mutation to prevent duplicates engines in the schedules
                       keep_elitism = 1, # Number of best solutions that are copied to the next generation
                       mutation_probability=MUTATION_PROBABILITY,
                    #    num_genes = NO_ENGINES,
                       initial_population=create_initial_population()  # Our custom create initial population function
                       )

/home/quintine/.pyenv/versions/3.11.5/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


In [20]:
# Run the GA instance for a specified number of generations
ga_instance.run()

# Retrieve the best solution found so far
solution, solution_fitness, _ = ga_instance.best_solution()

# Print or use the best solution
print("Best solution:", solution)
print("Best solution fitness:", solution_fitness)

Best solution: [ 92.  82.  36.  20.  49.  56.  91.  41.  37.  81. 100.  76.  77.  40.
  34.  64.  61.  24.  90.  66.  35.  68.  31.  42.]
Best solution fitness: 2515


# Check Solution

In [21]:
# Check duplicates
sorted_solution = sorted(solution)
sorted_engines_to_maintain = sorted(ENGINES_TO_MAINTAIN)

# Compare if both sorted lists are equal
print('The solution is a permutation (i.e., there are no duplicates): ', sorted_solution == sorted_engines_to_maintain)

The solution is a permutation (i.e., there are no duplicates):  True
